In [1]:
from copy import deepcopy
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import train_test_split

In [2]:
raw_train = pd.read_csv("Data/train.csv")
raw_test = pd.read_csv("Data/test.csv")
targets = ["max_price", "delta_abs", "delta_rel", "min_price"]


In [3]:
def shared_reformat(df):
    df = deepcopy(df)
    
    # create special targets (only on train set)
    if "max_price" in list(df):
        df["delta_abs"] = df["max_price"] - df["min_price"] 
        df["delta_rel"] = np.log(df["max_price"] / df["min_price"] )
    
    # deal with missing data
    #df.pixels_y = df.pixels_y.fillna(0)
    df.detachable_keyboard.fillna(0, inplace = True)  # only four cases. Assume none
    df.pixels_x.fillna(1920, inplace = True)   # only 2 cases, replace by most common value
    df.pixels_y.fillna(1080, inplace = True)   # only 2 cases, replace by most common value
    df.loc[df.screen_surface.isna(), "screen_surface"] = 'Glossy'       # 12 cases, large majority is Glossy,
    df.loc[df.screen_surface.isna() & df.pixels_x == 1920, "screen_surface"] = 'Matte'  # except here
    df.sort_values(by=['name'])
    df.cpu_details.fillna(method='bfill', inplace=True)  # take previous value, they are often similar
    df.gpu.fillna(method='bfill', inplace=True)  # take previous value, they are often similar
    df.weight.fillna(method='bfill', inplace=True)
    df.loc[df.os.isna() & df.brand == "Apple", "os"] = 'macOS'  # 1 case
    df.loc[df.os_details.isna() & df.brand == "Apple", "os_details"] = "macOS Mojave"  # 1 case, most common
    df.loc[df.os.isna() & df.brand == "Dell", "os"] = 'Windows'  # 1 case
    df.loc[df.os_details.isna() & df.brand == "Dell", "os_details"] = "Windows 10 Home"  # 1 case, most common
    

    # create new features
    df["cpu_brand"] = df.cpu.str.split(n=1).str[0]
    df["cpu_type"] = df.cpu.str.split(n=1).str[1]
    df["gpu_brand"] = df.gpu.str.split(n=1).str[0]
    df["gpu_series"] = df.gpu.str.split(n=2).str[1]
    df["os_type"] = df.os_details.str.split(n=1).str[1]
    df["os_nr"] = df.os_details.str.split(n=2).str[2]
    df["resolution"] = df.pixels_y / df.screen_size
    df["gimmick"] = df.detachable_keyboard + 5 * df.discrete_gpu + 3* df.touchscreen
    def get_speed(sentence):
        try:
            sentence = sentence.split()
            index = sentence.index('GHz')
            return pow(float(sentence[index-1]),2)
        except:
            return pow(2.4, 2)
    df["speed"] = df.cpu_details.apply(get_speed)
    def has_word(sentence, word):
        try:
            return word in sentence
        except:
            return False
    df["Dual-Core"] = df.cpu_details.apply(has_word, word="Dual-Core")
    df["Quad-Core"] = df.cpu_details.apply(has_word, word="Quad-Core")
    df["Hexa-Core"] = df.cpu_details.apply(has_word, word="Hexa-Core")
    df["Hyper-Threading"] = df.cpu_details.apply(has_word, word="Hyper-Threading")
    
    
    # correct a few mistakes in new features
    map_gpu_series = {"GeFoce":"GeForce", "RadeonÂ": "Radeon"}
    df.gpu_series = df.gpu_series.replace(map_gpu_series) 
    map_screen_surface = {"glossy":"Glossy", "matte":"Matte"}
    df.screen_surface = df.screen_surface.replace(map_screen_surface)  

    
    # make sure all values in categorical variables are strings
    cat_col = df.select_dtypes(include=['object']).columns
    df[cat_col] = df[cat_col].astype(str)
    
    # cleanup test set
    df.pixels_y = df.pixels_y.astype('int64')
    
    # shuffle the database
    df.sample(frac=1)
    return df


In [4]:
df_train = shared_reformat(raw_train)
df_test = shared_reformat(raw_test)

drop_cols=["id", "name", "base_name", "screen_size", "weight", "detachable_keyboard", "gpu_series",
              "ssd", "pixels_x", "cpu_details", "os_details", "cpu_brand", "cpu", "os", "min_price", "max_price", "delta_abs", "delta_rel"]

drop_cols_test=["id", "name", "base_name", "screen_size", "weight", "detachable_keyboard", "gpu_series",
              "ssd", "pixels_x", "cpu_details", "os_details", "cpu_brand", "cpu", "os"]
X = df_train.drop(columns=drop_cols, axis=1)
y = df_train[["min_price", "max_price"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(df_train.columns)
print(X.columns)
print(X.head(2))
X.dtypes

Index(['id', 'name', 'brand', 'base_name', 'screen_size', 'pixels_x',
       'pixels_y', 'screen_surface', 'touchscreen', 'cpu', 'cpu_details',
       'detachable_keyboard', 'discrete_gpu', 'gpu', 'os', 'os_details', 'ram',
       'ssd', 'storage', 'weight', 'min_price', 'max_price', 'delta_abs',
       'delta_rel', 'cpu_brand', 'cpu_type', 'gpu_brand', 'gpu_series',
       'os_type', 'os_nr', 'resolution', 'gimmick', 'speed', 'Dual-Core',
       'Quad-Core', 'Hexa-Core', 'Hyper-Threading'],
      dtype='object')
Index(['brand', 'pixels_y', 'screen_surface', 'touchscreen', 'discrete_gpu',
       'gpu', 'ram', 'storage', 'cpu_type', 'gpu_brand', 'os_type', 'os_nr',
       'resolution', 'gimmick', 'speed', 'Dual-Core', 'Quad-Core', 'Hexa-Core',
       'Hyper-Threading'],
      dtype='object')
    brand  pixels_y screen_surface  touchscreen  discrete_gpu  \
0  Lenovo      1080         Glossy            1             0   
1   Razer      1080          Matte            0             1   

  

C:\ProgramData\Anaconda3\envs\ComputerPriceRange\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


brand               object
pixels_y             int64
screen_surface      object
touchscreen          int64
discrete_gpu         int64
gpu                 object
ram                  int64
storage              int64
cpu_type            object
gpu_brand           object
os_type             object
os_nr               object
resolution         float64
gimmick            float64
speed              float64
Dual-Core             bool
Quad-Core             bool
Hexa-Core             bool
Hyper-Threading       bool
dtype: object

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_features = X.select_dtypes(['int64', 'float64']).columns

categorical_features = X.select_dtypes(include=['object']).columns

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer( 
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [6]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestRegressor())])

rf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [7]:
y_pred = rf.predict(X_test)
print(y_pred)

[[ 483.1704      505.0948    ]
 [1413.9811     1489.498     ]
 [1176.58696    1185.08562667]
 [ 365.8059      378.1962    ]
 [1548.4532     1656.9897    ]
 [1510.2946     1602.2882    ]
 [1647.8477     1719.0329    ]
 [ 160.8083      164.9055    ]
 [ 693.9881      733.9964    ]
 [1752.52646667 1830.60576667]
 [ 285.16105     289.10135   ]
 [ 579.8574      586.7328    ]
 [ 369.2488      372.5626    ]
 [ 725.62301667  734.27158333]
 [ 395.6556      404.8587    ]
 [1769.6872     1810.7174    ]
 [1781.8751     1806.1928    ]
 [ 503.2782      526.31525   ]
 [ 585.48996667  604.1749    ]
 [ 240.19426667  245.89786667]
 [ 418.0317      429.3401    ]
 [1683.4337     1702.0536    ]
 [1120.3135     1164.0087    ]
 [2409.8886     2447.0705    ]
 [1140.1606     1238.5982    ]
 [ 390.3219      410.29353333]
 [ 315.9069      343.1015    ]
 [1241.818      1274.7124    ]
 [ 159.2757      168.1888    ]
 [1600.17565    1702.9078    ]
 [1316.4197     1362.0404    ]
 [ 640.3009      650.5105    ]
 [1576.6

In [8]:
#def seppe_error(Y_true, Y_pred):
#    return (mean_absolute_error(Y_true.min_price, Y_pred[:,0]) + mean_absolute_error(Y_true.max_price, Y_pred[:,1])) / 2

#seppe_error(y_test, y_pred)

In [9]:
def seppe_error(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred, multioutput=[0.5, 0.5])

seppe_error(y_test, y_pred)

162.4526541067538

In [10]:
X_eval = df_test.drop(columns=drop_cols_test, axis=1)
print(X_eval.columns)
#
y_test_pred = rf.predict(X_eval)
X_train.columns

Index(['brand', 'pixels_y', 'screen_surface', 'touchscreen', 'discrete_gpu',
       'gpu', 'ram', 'storage', 'cpu_type', 'gpu_brand', 'os_type', 'os_nr',
       'resolution', 'gimmick', 'speed', 'Dual-Core', 'Quad-Core', 'Hexa-Core',
       'Hyper-Threading'],
      dtype='object')


Index(['brand', 'pixels_y', 'screen_surface', 'touchscreen', 'discrete_gpu',
       'gpu', 'ram', 'storage', 'cpu_type', 'gpu_brand', 'os_type', 'os_nr',
       'resolution', 'gimmick', 'speed', 'Dual-Core', 'Quad-Core', 'Hexa-Core',
       'Hyper-Threading'],
      dtype='object')

In [11]:
result = pd.DataFrame(y_test_pred)
result["id"] = df_test.id.values

result.head()

,0,1,id
0,1132.896800,1206.296700,28807
1,388.923300,409.383633,22559
2,801.741400,820.343800,28647
3,547.434433,562.953433,22141
4,2192.879300,2226.949000,26116


In [12]:
result.columns = ["min_price", "max_price", "ID"]
result.set_index(keys = 'ID')
result = result[[ "ID", "min_price", "max_price"]]
result.head()
result.to_csv('Data/jeroen_result.csv', index=False)

In [16]:
params = {
        'classifier__n_estimators':[200, 500,],
        'classifier__max_features' : [.15, .3, "auto", "log2", None],
        'classifier__max_samples' : [.1, .2, .5, None],
        'classifier__min_samples_leaf' : [1, 5, 10],
        'classifier__ccp_alpha' : [0, .01, .1, .2],
        'classifier__oob_score' : [True, False]
    }
scoring = {'Seppe error': make_scorer(seppe_error, greater_is_better=False)}
CV = GridSearchCV(rf, params, scoring=make_scorer(seppe_error, greater_is_better=False), cv=3,n_jobs= -1)
                  
CV.fit(X_train, y_train)
print(CV.best_params_)    
print(CV.best_score_)

{'classifier__ccp_alpha': 0.01, 'classifier__max_features': 0.15, 'classifier__max_samples': None, 'classifier__min_samples_leaf': 1, 'classifier__n_estimators': 500, 'classifier__oob_score': True}
-151.45903496625303
